<a href="https://colab.research.google.com/github/ms2288/ds-section2-sprint1/blob/main/n213-ridge-regression/n213a-ridge-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 1 / NOTE 3*

# 📝 Assignment
---

# Ridge Regression

이번 모듈에서 배운 내용을 조금 다른 데이터를 통해 복습해 봅시다. 데이터에 대한 정보는 다음 링크를 참조해 주세요.

- [Melbourne Housing Market]('https://www.kaggle.com/anthonypino/melbourne-housing-market')

이번에도 Price 변수를 target으로 정하고 모델을 구성해 봅시다.

In [9]:
pip install category-encoders

     |████████████████████████████████| 80 kB 3.6 MB/s 


In [30]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

# get data
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/melbourne_house_prices/MELBOURNE_HOUSE_PRICES_LESS.csv')

# drop high cardinality cols
df.drop(columns=['Suburb','Address','SellerG','Date'], inplace=True)

# drop na values in Price
df.dropna(subset=['Price'], inplace=True)

# drop duplicated rows
df.drop_duplicates(inplace=True)



In [31]:
df

,Rooms,Type,Price,Method,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,3,h,1490000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,3,h,1220000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,3,h,1420000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,3,h,1515000.0,S,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,2,h,670000.0,S,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council
...,...,...,...,...,...,...,...,...,...
63015,2,u,340000.0,SP,3199,South-Eastern Metropolitan,17055,38.0,Frankston City Council
63016,2,u,347700.0,SP,3199,South-Eastern Metropolitan,17055,38.0,Frankston City Council
63017,3,h,808000.0,S,3072,Northern Metropolitan,14577,8.4,Darebin City Council
63018,3,h,566000.0,S,3064,Northern Metropolitan,5833,20.6,Hume City Council


### 1) 데이터의 특성을 파악하기 위해 pandas_profiling을 사용해 보세요.
- **범주형 특성으로 원핫인코딩을 할 수 있는 특성의 전체 개수를 과제 제출폼에 제출하세요.**

In [32]:
## import OneHotEncoder
from category_encoders import OneHotEncoder

## 원핫 인코딩
encoder = OneHotEncoder(use_cat_names = True)
df = encoder.fit_transform(df)
df

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Rooms,Type_h,Type_t,Type_u,Price,Method_S,Method_SP,Method_VB,Method_PI,Method_SA,Postcode,Regionname_Northern Metropolitan,Regionname_Western Metropolitan,Regionname_Southern Metropolitan,Regionname_South-Eastern Metropolitan,Regionname_Eastern Metropolitan,Regionname_Northern Victoria,Regionname_Eastern Victoria,Regionname_Western Victoria,Propertycount,Distance,CouncilArea_Yarra City Council,CouncilArea_Moonee Valley City Council,CouncilArea_Port Phillip City Council,CouncilArea_Brimbank City Council,CouncilArea_Darebin City Council,CouncilArea_Hobsons Bay City Council,CouncilArea_Stonnington City Council,CouncilArea_Boroondara City Council,CouncilArea_Monash City Council,CouncilArea_Kingston City Council,CouncilArea_Maroondah City Council,CouncilArea_Bayside City Council,CouncilArea_Banyule City Council,CouncilArea_Glen Eira City Council,CouncilArea_Whitehorse City Council,CouncilArea_Maribyrnong City Council,CouncilArea_Hume City Council,CouncilArea_Moreland City Council,CouncilArea_Manningham City Council,CouncilArea_Melbourne City Council,CouncilArea_Melton City Council,CouncilArea_Frankston City Council,CouncilArea_Casey City Council,CouncilArea_Greater Dandenong City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Whittlesea City Council,CouncilArea_Knox City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra Ranges Shire Council,CouncilArea_Cardinia Shire Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Mitchell Shire Council,CouncilArea_Moorabool Shire Council,CouncilArea_Murrindindi Shire Council
0,3,1,0,0,1490000.0,1,0,0,0,0,3067,1,0,0,0,0,0,0,0,4019,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,1,0,0,1220000.0,1,0,0,0,0,3067,1,0,0,0,0,0,0,0,4019,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1,0,0,1420000.0,1,0,0,0,0,3067,1,0,0,0,0,0,0,0,4019,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,1515000.0,1,0,0,0,0,3040,0,1,0,0,0,0,0,0,1543,7.5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,1,0,0,670000.0,1,0,0,0,0,3042,0,1,0,0,0,0,0,0,3464,10.4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63015,2,0,0,1,340000.0,0,1,0,0,0,3199,0,0,0,1,0,0,0,0,17055,38.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
63016,2,0,0,1,347700.0,0,1,0,0,0,3199,0,0,0,1,0,0,0,0,17055,38.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
63017,3,1,0,0,808000.0,1,0,0,0,0,3072,1,0,0,0,0,0,0,0,14577,8.4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63018,3,1,0,0,566000.0,1,0,0,0,0,3064,1,0,0,0,0,0,0,0,5833,20.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
target='Price'
X = df.drop([target],axis=1)
y = df[target]

### 2) 범주형 변수들에 대해 one-hot encoding(category_encoders)을 수행하고 데이터를 train/test 셋으로 무작위로 나누어 주세요.(train/test = 80%/20%)
- [category_encoders.one_hot.OneHotEncoder](https://contrib.scikit-learn.org/category_encoders/onehot.html#one-hot), 파라미터는 use_cat_names = True 사용
- 데이터 분리는 [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn-model-selection-train-test-split) 을 사용해 주세요. 사용해야 하는 파라미터는 test_size, train_size, random_state=2 입니다. 나머지는 default 사용합니다.
- **분리된 학습데이터의 첫번째 샘플의 인덱스 값를 과제 제출폼에 제출하세요.**


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,train_size=0.8,random_state=2)

In [35]:
X_train

,Rooms,Type_h,Type_t,Type_u,Method_S,Method_SP,Method_VB,Method_PI,Method_SA,Postcode,Regionname_Northern Metropolitan,Regionname_Western Metropolitan,Regionname_Southern Metropolitan,Regionname_South-Eastern Metropolitan,Regionname_Eastern Metropolitan,Regionname_Northern Victoria,Regionname_Eastern Victoria,Regionname_Western Victoria,Propertycount,Distance,CouncilArea_Yarra City Council,CouncilArea_Moonee Valley City Council,CouncilArea_Port Phillip City Council,CouncilArea_Brimbank City Council,CouncilArea_Darebin City Council,CouncilArea_Hobsons Bay City Council,CouncilArea_Stonnington City Council,CouncilArea_Boroondara City Council,CouncilArea_Monash City Council,CouncilArea_Kingston City Council,CouncilArea_Maroondah City Council,CouncilArea_Bayside City Council,CouncilArea_Banyule City Council,CouncilArea_Glen Eira City Council,CouncilArea_Whitehorse City Council,CouncilArea_Maribyrnong City Council,CouncilArea_Hume City Council,CouncilArea_Moreland City Council,CouncilArea_Manningham City Council,CouncilArea_Melbourne City Council,CouncilArea_Melton City Council,CouncilArea_Frankston City Council,CouncilArea_Casey City Council,CouncilArea_Greater Dandenong City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Whittlesea City Council,CouncilArea_Knox City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra Ranges Shire Council,CouncilArea_Cardinia Shire Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Mitchell Shire Council,CouncilArea_Moorabool Shire Council,CouncilArea_Murrindindi Shire Council
602,4,1,0,0,1,0,0,0,0,3106,0,0,0,0,1,0,0,0,6202,15.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40851,5,1,0,0,1,0,0,0,0,3169,0,0,0,1,0,0,0,0,2727,17.5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
57233,4,1,0,0,0,0,1,0,0,3028,0,1,0,0,0,0,0,0,7630,15.5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8816,4,1,0,0,1,0,0,0,0,3033,0,1,0,0,0,0,0,0,5629,11.7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3481,3,1,0,0,1,0,0,0,0,3076,1,0,0,0,0,0,0,0,10926,19.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44542,3,1,0,0,1,0,0,0,0,3173,0,0,0,1,0,0,0,0,8459,25.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
43399,2,0,0,1,1,0,0,0,0,3088,1,0,0,0,0,0,0,0,8524,16.1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8789,6,1,0,0,0,0,0,1,0,3037,0,1,0,0,0,0,0,0,5556,18.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51700,3,1,0,0,0,0,0,1,0,3047,1,0,0,0,0,0,0,0,4294,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 3) SelectKBest를 사용해서 20개의 특성만 selection 하세요
- score_func 에는 f_regression을 사용합니다
- **가장 스코어(Attributes: scores_ 사용)가 높게 나온 특성의 이름을 과제 제출폼에 제출하세요.**

In [38]:
# target(Price)와 가장 correlated 된 features 를 k개 고르는 것이 목표입니다.

## f_regresison, SelectKBest
from sklearn.feature_selection import f_regression, SelectKBest

## selctor 정의합니다.
selector = SelectKBest(score_func=f_regression, k=20)

## 학습데이터에 fit_transform 
X_train_selected = selector.fit_transform(X_train, y_train)

## 테스트 데이터는 transform
X_test_selected = selector.transform(X_test)


all_names = X_train.columns
all_names
# X_train_selected.shape, X_test_selected.shape

Index(['Rooms', 'Type_h', 'Type_t', 'Type_u', 'Method_S', 'Method_SP',
       'Method_VB', 'Method_PI', 'Method_SA', 'Postcode',
       'Regionname_Northern Metropolitan', 'Regionname_Western Metropolitan',
       'Regionname_Southern Metropolitan',
       'Regionname_South-Eastern Metropolitan',
       'Regionname_Eastern Metropolitan', 'Regionname_Northern Victoria',
       'Regionname_Eastern Victoria', 'Regionname_Western Victoria',
       'Propertycount', 'Distance', 'CouncilArea_Yarra City Council',
       'CouncilArea_Moonee Valley City Council',
       'CouncilArea_Port Phillip City Council',
       'CouncilArea_Brimbank City Council', 'CouncilArea_Darebin City Council',
       'CouncilArea_Hobsons Bay City Council',
       'CouncilArea_Stonnington City Council',
       'CouncilArea_Boroondara City Council',
       'CouncilArea_Monash City Council', 'CouncilArea_Kingston City Council',
       'CouncilArea_Maroondah City Council',
       'CouncilArea_Bayside City Council', 'Coun

### 4) RidgeCV를 사용해서 회귀모델을 만들어 보세요
- attribute는 normalize=True, cv=5를 사용하세요. 나머지는 default
- **alphas = [0, 0.001, 0.01, 0.1, 1] 을 시도해보고 가장 좋은 점수가 나온 alpha 값을 과제 제출폼에 제출하세요.**

In [ ]:
# 코랩에서 아래와 같은 에러 발생시, sklearn 재설치 진행합니다.
# 에러메시지: 'RidgeCV' object has no attribute 'best_score_' (ver.0.23부터 적용됨. 공식문서에서 확인 가능.)
# 해결: sklearn 재설치 필요. 버전 0.23 이상
# 재설치 방법⇣: 

# sklearn 버전 확인 -> 0.23보다 낮다면 재설치 진행.
import sklearn
sklearn.__version__

# sklearn 재설치
!pip uninstall scikit-learn -y
!pip install -U scikit-learn

# sklearn 버전 재확인 -> 0.23 이상으로 설치되었는지 확인
import sklearn
sklearn.__version__

In [ ]:
### 이곳에서 과제를 진행해 주세요 ###

### 5) 테스트셋에 대한 MAE 값을 계산하고 모델이 잘 만들어졌는지 확인해 보세요.
- **테스트셋에 대한 MAE 값을 과제 제출폼에 제출하세요.**

In [ ]:
### 이곳에서 과제를 진행해 주세요 ###

## 🔥 도전과제

### 6) best ridge regression model을 만드세요.
    - 특성의 갯수에 제한이 없습니다.
    - 다항회귀를 사용하세요
    - 여러 람다값을 시도하세요
    - 람다 0인 경우를 비교하세요
    - RidgeCV를 사용하세요
    - 만든 모델의 특징을 간략히 설명하고 테스트세트 성능 결과를 메신저에서 공유하세요

In [ ]:
### 이곳에서 과제를 진행해 주세요 ###